In [14]:
import tinyik
arm = tinyik.Actuator(['x', [1., 0., 0.], 'y', [0, 1., 0.]])

In [18]:
arm.angles

array([0.98279372, 0.        ])

In [16]:
arm.ee = [1,2,3]

In [23]:
import numpy as np

# Define robot dimensions (link lengths)
link1_length = 0.2  # Length of the first link
link2_length = 0.1  # Length of the second link
link3_length = 0.1  # Length of the third link

# Define the end-effector position you want to reach
desired_position = np.array([0.2, 0.1, 0.15])  # [x, y, z]

# Set the maximum number of iterations and a tolerance for convergence
max_iterations = 1000
tolerance = 1e-6

# Initialize joint angles
theta1 = 0.0  # Initial guess for the first joint angle (yaw)
theta2 = 0.0  # Initial guess for the second joint angle (pitch)
theta3 = 0.0  # Initial guess for the third joint angle (roll)

# Define the IK solver loop
for i in range(max_iterations):
    # Calculate the current end-effector position based on the current joint angles
    x = link1_length * np.cos(theta1) + link2_length * np.cos(theta1 + theta2) + link3_length * np.cos(theta1 + theta2 + theta3)
    y = link1_length * np.sin(theta1) + link2_length * np.sin(theta1 + theta2) + link3_length * np.sin(theta1 + theta2 + theta3)
    z = 0.0  # Assuming a planar robot in the XY plane

    # Calculate the position error
    error = desired_position - np.array([x, y, z])

    # Check if the error is within tolerance
    if np.linalg.norm(error) < tolerance:
        print("IK converged in", i, "iterations")
        break

    # Calculate the Jacobian matrix (partial derivatives) to update joint angles
    J = np.array([
        [-link1_length * np.sin(theta1) - link2_length * np.sin(theta1 + theta2) - link3_length * np.sin(theta1 + theta2 + theta3),
         -link2_length * np.sin(theta1 + theta2) - link3_length * np.sin(theta1 + theta2 + theta3),
         -link3_length * np.sin(theta1 + theta2 + theta3)],
        [link1_length * np.cos(theta1) + link2_length * np.cos(theta1 + theta2) + link3_length * np.cos(theta1 + theta2 + theta3),
         link2_length * np.cos(theta1 + theta2) + link3_length * np.cos(theta1 + theta2 + theta3),
         link3_length * np.cos(theta1 + theta2 + theta3)],
        [0, 0, 0]
    ])

    # Update joint angles using the Jacobian
    joint_angles = np.array([theta1, theta2, theta3]) + np.linalg.pinv(J).dot(error)

    theta1, theta2, theta3 = joint_angles

# Output the final joint angles
print("Final Joint Angles (yaw, pitch, roll):", theta1, theta2, theta3)


Final Joint Angles (yaw, pitch, roll): -5.933538350255106 8.538736530101744 9.763719434558285
